In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import sys
import re
import os
from os import listdir
import matplotlib.pyplot as plt
from os.path import isfile, join
from sklearn.model_selection import train_test_split
import numpy as np

import librosa
import librosa.display

import cv2
import zipfile
from PIL import Image

import time
import copy

# Get accent archive

In [2]:
%cd /content/drive/MyDrive/NIS_2022/accent_archive

/content/drive/MyDrive/NIS_2022/accent_archive


In [3]:
!git clone https://github.com/akshanshchaudhry/Speech-Accent-Recognition.git

Cloning into 'Speech-Accent-Recognition'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), done.


In [13]:
!pip install pydub

In [32]:
%cd ../Speech-Accent-Recognition/src

/content/drive/MyDrive/NIS_2022/accent_archive/Speech-Accent-Recognition/src


In [25]:
!python fromwebsite.py /content/drive/MyDrive/NIS_2022/accent_archive/data/bio_metadata.csv  chinese english russian japanese arabic spanish korean

downloading from http://accent.gmu.edu/browse_language.php?function=find&language=japanese
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=221
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=222
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=223
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=224
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=225
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=226
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=227
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=486
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=543
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=827
downloading from ht

In [63]:
df = pd.read_csv ('/content/drive/MyDrive/NIS_2022/accent_archive/data/processed_bio_metadata.csv',
                  sep='\t')

In [43]:
categories = []
for index, row in df.iterrows():
    category = 'default'
    if 'china' in row['birth_place'] or 'taiwan' in row['birth_place']:
        category = 'chinese'
    elif 'english' in row['language_num']:
        if 'usa' in row['birth_place']:
            category = 'usa'
        elif 'uk' in row['birth_place']:
            category = 'uk'
        elif 'canada' in row['birth_place']:
            category = 'canada'
        elif 'australia' in row['birth_place']:
            category = 'australia'
    elif 'russian' in row['language_num']:
        category = 'russian'
    elif 'arabic' in row['language_num']:
        category = 'arabic'
    elif 'spanish' in row['language_num']:
        category = 'spanish'
    elif 'korean' in row['language_num']:
        category = 'korean'
    elif 'japanese' in row['language_num']:
        category = 'japanese'
    categories.append(category)
    

In [52]:
df['category'] = categories

In [53]:
len(categories)

1521

In [54]:
df.to_csv(r'/content/drive/MyDrive/NIS_2022/accent_archive/data/processed_bio_metadata.csv', index=False, sep='\t', header='true')

In [68]:
!python getaudio.py /content/drive/MyDrive/NIS_2022/accent_archive/data/processed_bio_metadata.csv

# Dataset create

In [5]:
def get_melspectrogram_db(file_path, sr=None, n_fft=2048, hop_length=512, n_mels=128, fmin=20, fmax=8300, top_db=80):
    path = os.fspath(file_path)
    wav,sr = librosa.load(path,sr=sr)
    # if wav.shape[0]<5*sr:
    #     wav=np.pad(wav,int(np.ceil((5*sr-wav.shape[0])/2)),mode='reflect')
    # else:
    #     wav=wav[:5*sr]
    spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft,
                hop_length=hop_length,n_mels=n_mels,fmin=fmin,fmax=fmax)
    spec_db=librosa.power_to_db(spec,top_db=top_db)
    return spec_db


def spec_to_image(spec, eps=1e-6):
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)
    return spec_scaled

In [6]:
audio_dir = '/content/drive/MyDrive/NIS_2022/accent_archive/data/audio/'
mel_spectrograms_dir = '/content/drive/MyDrive/NIS_2022/accent_archive/data/mel_spectrograms/'

train_dir = mel_spectrograms_dir + 'train'
valid_dir = mel_spectrograms_dir + 'validation'

In [8]:
for subdir, dirs, files in os.walk(audio_dir):
    if len(files) == 0: 
        continue
    accent = os.path.basename(subdir)
    train, test = train_test_split(files, test_size=0.2)
    for audio_file in train:
        audio_path = os.path.join(subdir, audio_file)
        try:
          melspectrogram = spec_to_image(get_melspectrogram_db(audio_path))
        except:
            continue
        img = Image.fromarray(melspectrogram)
        save_path = train_dir + '/' + accent
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        img.save(save_path+'/'+os.path.splitext(os.path.basename(audio_path))[0]+'.png')

    for audio_file in test:
        audio_path = os.path.join(subdir, audio_file)
        try:
          melspectrogram = spec_to_image(get_melspectrogram_db(audio_path))
        except:
            continue
        img = Image.fromarray(melspectrogram)
        save_path = valid_dir + '/' + accent
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        img.save(save_path+'/'+os.path.splitext(os.path.basename(audio_path))[0]+'.png')

# Data loading

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torchaudio

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

import torchvision.models as models

In [253]:
input_size = 224
batch_size = 8

In [254]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

mel_spectrograms_dir = '/content/drive/MyDrive/NIS_2022/accent_archive/data/mel_spectrograms/'
image_datasets = {x: ImageFolder(os.path.join(mel_spectrograms_dir, x), data_transforms[x]) for x in ['train', 'validation']}

In [255]:
def make_weights_for_balanced_classes(images, nclasses):                        
    count = [0] * nclasses                                                      
    for item in images:                                                         
        count[item[1]] += 1                                                     
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))                                                   
    for i in range(nclasses):                                                   
        weight_per_class[i] = N/float(count[i])                                 
    weight = [0] * len(images)                                              
    for idx, val in enumerate(images):                                          
        weight[idx] = weight_per_class[val[1]]                                  
    return weight                         

In [256]:
# weights = make_weights_for_balanced_classes(image_datasets['train'].imgs, len(image_datasets['train'].classes))                                                                
# weights = torch.DoubleTensor(weights)                                       
# sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights)) 

In [257]:
# dataloaders_dict = {}
# dataloaders_dict['train'] = torch.utils.data.DataLoader(image_datasets['train'], 
#                                                    batch_size=batch_size, 
#                                                   #  shuffle=True, 
#                                                    sampler=sampler,
#                                                    num_workers=4) 

In [258]:
# dataloaders_dict['validation'] = torch.utils.data.DataLoader(image_datasets['validation'], 
#                                                    batch_size=batch_size, 
#                                                    shuffle=True, 
#                                                    num_workers=4) 

In [259]:
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, 
                                                   num_workers=4) for x in ['train', 'validation']}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [260]:
num_classes = 10
num_epochs = 15

In [261]:
resnet18_pretrained = models.resnet18(pretrained=True)
resnet18_pretrained.fc = nn.Linear(512, num_classes)

In [262]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [263]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'validation':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [264]:
resnet18_pretrained = resnet18_pretrained.to(device)

params_to_update = resnet18_pretrained.parameters()
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [265]:
criterion = nn.CrossEntropyLoss()
# Train and evaluate
resnet18_pretrained, hist = train_model(resnet18_pretrained,
                             dataloaders_dict,
                             criterion, optimizer_ft, 
                             num_epochs=num_epochs)

Epoch 0/14
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 2.0793 Acc: 0.2935
validation Loss: 1.9866 Acc: 0.3503

Epoch 1/14
----------
train Loss: 1.6154 Acc: 0.4606
validation Loss: 2.4215 Acc: 0.2347

Epoch 2/14
----------
train Loss: 1.1575 Acc: 0.6234
validation Loss: 2.2836 Acc: 0.3265

Epoch 3/14
----------
train Loss: 0.7123 Acc: 0.7818
validation Loss: 2.2637 Acc: 0.3571

Epoch 4/14
----------
train Loss: 0.4234 Acc: 0.8935
validation Loss: 2.5840 Acc: 0.3605

Epoch 5/14
----------
train Loss: 0.2250 Acc: 0.9498
validation Loss: 2.6697 Acc: 0.3265

Epoch 6/14
----------
train Loss: 0.2189 Acc: 0.9429
validation Loss: 2.9263 Acc: 0.3231

Epoch 7/14
----------
train Loss: 0.1858 Acc: 0.9506
validation Loss: 2.5619 Acc: 0.3129

Epoch 8/14
----------
train Loss: 0.1853 Acc: 0.9576
validation Loss: 2.8874 Acc: 0.3401

Epoch 9/14
----------
train Loss: 0.1916 Acc: 0.9446
validation Loss: 2.7654 Acc: 0.3571

Epoch 10/14
----------
train Loss: 0.1346 Acc: 0.9714
validation Loss: 2.8042 Acc: 0.3537

Epoch 11/14
----------
train Lo

In [266]:
from sklearn.metrics import confusion_matrix, classification_report
from itertools import chain

y_pred_list = []
y_labels_list = []
resnet18_pretrained.eval()
with torch.no_grad():
    for batch in dataloaders_dict['validation']:
        images, labels = batch
        y_labels_list.append(labels.numpy())
        images, _ = images.cuda(), labels.cuda()
        y_test_pred = resnet18_pretrained(images)
        y_pred_softmax = torch.log_softmax(y_test_pred, dim = 1)
        _, y_pred_tags = torch.max(y_pred_softmax, dim = 1) 
        y_pred_list.append([x for x in y_pred_tags.cpu().numpy()])

y_pred_list = list(chain.from_iterable(y_pred_list))
y_labels_list = list(chain.from_iterable(y_labels_list))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [267]:
print(classification_report(y_labels_list, y_pred_list))

              precision    recall  f1-score   support

           0       0.19      0.17      0.18        40
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00        10
           3       0.52      0.33      0.40        43
           4       0.00      0.00      0.00         9
           5       0.20      0.15      0.17        20
           6       0.00      0.00      0.00        17
           7       0.29      0.49      0.36        47
           8       0.00      0.00      0.00        14
           9       0.50      0.73      0.59        85

    accuracy                           0.37       294
   macro avg       0.17      0.19      0.17       294
weighted avg       0.30      0.37      0.32       294



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [268]:
confusion_matrix(y_labels_list, y_pred_list)

array([[ 7,  0,  0,  3,  0,  4,  1, 15,  0, 10],
       [ 2,  0,  0,  0,  0,  1,  0,  2,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  2,  0,  8],
       [ 7,  0,  0, 14,  1,  3,  0, 12,  0,  6],
       [ 0,  0,  0,  2,  0,  1,  1,  3,  0,  2],
       [ 3,  0,  1,  1,  0,  3,  0,  3,  0,  9],
       [ 1,  0,  1,  0,  0,  0,  0,  7,  0,  8],
       [ 4,  1,  1,  4,  0,  0,  1, 23,  0, 13],
       [ 3,  0,  0,  0,  0,  1,  1,  6,  0,  3],
       [10,  0,  1,  3,  0,  2,  0,  7,  0, 62]])

In [269]:
dataloaders_dict['train'].dataset.classes

['arabic',
 'australia',
 'canada',
 'chinese',
 'japanese',
 'korean',
 'russian',
 'spanish',
 'uk',
 'usa']